In [107]:
!pip install bertopic[visulaization]

In [108]:
import numpy as np
import pandas as pd
from copy import deepcopy
from bertopic import BERTopic

In [109]:
emails = pd.read_csv('C:\Project DBs\Email\emails.csv')
#email_subset = emails[:10000]
email_subset = emails.sample(frac=0.02, random_state=1)

print(email_subset.shape)
print(email_subset.head())

(10348, 2)
                              file  \
186822  jones-t/all_documents/634.   
308790  mann-k/all_documents/5690.   
82383         dasovich-j/sent/423.   
227299          kaminski-v/var/63.   
301824     mann-k/_sent_mail/3208.   

                                                  message  
186822  Message-ID: <17820178.1075846925335.JavaMail.e...  
308790  Message-ID: <29110382.1075845717882.JavaMail.e...  
82383   Message-ID: <6812040.1075843194135.JavaMail.ev...  
227299  Message-ID: <21547648.1075856642126.JavaMail.e...  
301824  Message-ID: <12684200.1075846107179.JavaMail.e...  


In [110]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [111]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        
    }

In [112]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('');
            
        else:
            results.append(email[key])
    return results

In [113]:
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

def filter_words(texts):
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [114]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head(3))

                                                body
0  It would be nice if you could be at my dinner,...
1  Absolutely.Good point!  Can Peter start to dra...
2  My apologies.  My schedule melted down after w...


In [115]:
gmail_data=pd.read_csv('C:\\Users\\user\\Downloads\\MailBodyOutput2.csv')
gmail_data.dropna(inplace=True)
gmail_data.head()

,ID,body,to,from_
0,A1,"Hi Dmml,\r\nWelcome to Google. Your new accoun...","('dmml2test@gmail.com',)",googlecommunityteam-noreply@google.com
1,A2,"Your impartial, comprehensive news source.\r\n...","('dmml2test@gmail.com',)",DailyDigest@email.join1440.com
4,A5,** Important: Confirm Your Subscription to Nex...,"('dmml2test@gmail.com',)",dave@davenetics.com
5,A6,Your NextDraft subscription has been confirmed...,"('dmml2test@gmail.com',)",dave@davenetics.com
6,A7,Welcome to ELEVATOR.\r\n\r\nYour inbox is like...,"('dmml2test@gmail.com',)",yo@gettheelevator.com


In [116]:
email_df=email_df.append(gmail_data.drop('ID',axis=1))

In [117]:
#docs = list(email_df.loc[:, "body"].values)

In [118]:
docs=filter_words(docs)

In [119]:
doc=[]
for x in docs:
    doc.append(' '.join(x))

In [120]:
doc[:5]

['would nice could dinner since probably knowanyone else anytime want go lunch check house status glad go',
 'absolutely good point peter start draft override letter kay mann enrondieball john grigby reminder us need form override letter go withthe form turbine contract kay',
 'apologies schedule melted talked monday swhere folks came concern size supposed beno larger lobbied aceves apparently ok oversized group folks group talked himoriginally pretty sure five violate rules folks wonderedif groups smaller could hook upwith sorry wrinkle think wespoke gets real ugly trying find smaller group let know fortunately another team case due two weeks best jeff',
 'vince uk var breached limit last week uk traders asked us review correlations across uk gas power aswell correlations across efa slots part work last week update correlations based historical prices tanya richard lewisleppard lon ect ect rudy dautel hou ect ect kirstee hewitt lon ect ect naveen andrews corp enron enron david port ma

In [121]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

In [122]:
model = BERTopic(language="english",embedding_model=sentence_model,nr_topics=5)

In [123]:
topics, probs = model.fit_transform(doc)

In [124]:
model.get_topic_freq()

,Topic,Count
0,-1,9180
1,0,944
2,1,809
3,2,675
4,3,386
5,4,330


In [125]:
model.get_topic(0)

[('enron', 0.294916734145288),
 ('com', 0.27562194575188476),
 ('dec', 0.024106695592577075),
 ('active', 0.022692633364318728),
 ('mark', 0.02163497498458607),
 ('john', 0.019785524444102497),
 ('prc', 0.019105639067446223),
 ('davis', 0.016538994627598086),
 ('mail', 0.015807781871377213),
 ('michael', 0.015151113567478036)]

In [126]:
model.get_topic(2)

[('message', 0.054519039661217836),
 ('original', 0.05271421258147388),
 ('get', 0.03475588384938338),
 ('know', 0.032454658311676834),
 ('going', 0.031616503520143086),
 ('go', 0.02988355529664528),
 ('good', 0.026091687562119704),
 ('see', 0.025367201717068227),
 ('back', 0.023937225806053588),
 ('love', 0.02379265801812085)]

In [127]:
model.visualize_topics()

In [128]:
#new_topics, new_probs=model.reduce_topics(docs, topics, probs, nr_topics=20)

In [129]:
model.get_topic_info()

,Topic,Count,Name
0,-1,9180,-1_com_enron_https_px
1,0,944,0_enron_com_dec_active
2,1,809,1_link_post_name_add
3,2,675,2_message_original_get_know
4,3,386,3_name_like_thank_signature
5,4,330,4_magnet_lead_name_link


In [143]:
model.find_topics('subscribe')

([1, 3, 2, 4, -1],
 [0.5122292097298022,
  0.5095862124695291,
  0.4597678779246596,
  0.4469649311815091,
  0.42869310539136735])

In [146]:
model.visualize_barchart()


In [147]:
model.visualize_heatmap()

In [131]:
topics_to_assign=[]
for item in gmail_data.iterrows():
    assigned_topic,prob=model.transform(item[1]['body'])
    topics_to_assign.append(assigned_topic[0])
gmail_data['Topic']=topics_to_assign

C:\Users\user\anaconda3\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [132]:
gmail_data.head()

,ID,body,to,from_,Topic
0,A1,"Hi Dmml,\r\nWelcome to Google. Your new accoun...","('dmml2test@gmail.com',)",googlecommunityteam-noreply@google.com,-1
1,A2,"Your impartial, comprehensive news source.\r\n...","('dmml2test@gmail.com',)",DailyDigest@email.join1440.com,-1
4,A5,** Important: Confirm Your Subscription to Nex...,"('dmml2test@gmail.com',)",dave@davenetics.com,-1
5,A6,Your NextDraft subscription has been confirmed...,"('dmml2test@gmail.com',)",dave@davenetics.com,-1
6,A7,Welcome to ELEVATOR.\r\n\r\nYour inbox is like...,"('dmml2test@gmail.com',)",yo@gettheelevator.com,-1


In [138]:
gmail_data.loc[ gmail_data['Topic'] == -1, 'Topic'] = np.unique(gmail_data['Topic']).max()+1

In [148]:
gmail_data.to_csv('Clustered_Mails_BERTopic.csv',encoding='utf-8')

In [ ]:
model.save("Bert_model")

ANN

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
import keras.optimizers

In [ ]:
np.unique(gmail_data['Topic'])

In [ ]:
gmail_data.head()

In [150]:
data=pd.read_csv("C:/Users/user/Downloads/MailOutput_combined.csv")

In [151]:
data.dropna(inplace=True)

In [152]:
data=pd.merge(gmail_data[['ID','Topic']],data.drop('Topic',axis=1),on="ID")
data.to_csv('Clustered_Mails_BERTopic_Final.csv',encoding='utf-8')

In [ ]:
data.drop(['Date','Sender','Subject','ID'],axis=1,inplace=True)

In [153]:
data.shape

(1976, 17)

In [154]:
data.head()

,ID,Topic,Date,Sender,Subject,EmailSize,Status,isSubscription,isEncrypted,isFlagged,AttachmentCount,AttachmentSize,isPrimaryRecipient,NoOfToValues,isCC,NoOfCCValues,isBCC
0,A1,5,2022-03-10 09:50:34-08:00,googlecommunityteam-noreply@google.com,"Dmml, finish setting up your new Google Account",81274,True,False,False,False,0,0,True,1,False,0,False
1,A2,5,2022-03-11 06:46:10+11:00,DailyDigest@email.join1440.com,You're in! Here's how 1440 works...,36157,False,True,False,False,0,0,True,1,False,0,False
2,A5,5,2022-03-10 19:49:16+00:00,dave@davenetics.com,Verify Your Email Address for NextDraft,23484,False,False,False,False,0,0,True,1,False,0,False
3,A6,5,2022-03-10 19:50:00+00:00,dave@davenetics.com,NextDraft: Subscription Confirmed,23434,False,False,False,False,0,0,True,1,False,0,False
4,A7,5,2022-03-11 06:51:13+11:00,yo@gettheelevator.com,Welcome to ELEVATOR,29669,False,True,False,False,0,0,True,1,False,0,False


In [155]:
data.dtypes

ID                    object
Topic                  int64
Date                  object
Sender                object
Subject               object
EmailSize              int64
Status                  bool
isSubscription          bool
isEncrypted             bool
isFlagged               bool
AttachmentCount        int64
AttachmentSize         int64
isPrimaryRecipient      bool
NoOfToValues           int64
isCC                    bool
NoOfCCValues           int64
isBCC                   bool
dtype: object

In [156]:
data=pd.read_csv("C:/Users/user/Downloads/MailOutput_combined.csv")

In [ ]:
y=data['Topic']
X=data.drop('Topic',axis=1)

In [ ]:
#encoder = LabelEncoder()
#encoder.fit(y)
#encoded_Y = encoder.transform(y)
# convert integers to dummy variables (one hot encoded)
#dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
#dummy_y.shape

In [ ]:
y_cat = tf.keras.utils.to_categorical(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size = 0.2, random_state = 0)

In [ ]:
#Performing Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    staircase=True,
    decay_steps=10000,
    decay_rate=0.9)

In [ ]:
ann_classifier = Sequential()

early_stopping_cb = keras.callbacks.EarlyStopping(monitor="accuracy", patience=25)
# Hidden layers
ann_classifier.add(Dense(units = 54, activation = 'relu',kernel_initializer='random_normal', bias_initializer='zeros'))

ann_classifier.add(Dense(units=15, activation='relu',kernel_initializer='random_normal',bias_initializer='zeros'))

ann_classifier.add(tf.keras.layers.Dropout(rate=0.25))

ann_classifier.add(Dense(units = 15, activation = 'relu',kernel_regularizer=tf.keras.regularizers.l2(l=0.01),kernel_initializer='random_normal',
    bias_initializer='zeros'))

# Output layer
ann_classifier.add(Dense(units = len(np.unique(y)), activation = 'softmax'))


# Compiling the ANN
ann_classifier.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule) , loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="auto",
    name="categorical_crossentropy",
), metrics = ['accuracy'])



In [ ]:

# Fitting the ANN to the training set
ann_classifier.fit(X_train, y_train, batch_size = 8, epochs = 1000,callbacks=[early_stopping_cb])
